In [1]:
## setup libs 

pkg <- c("rtracklayer", "tictoc","here", "tidyverse", "BiocParallel")

invisible(suppressMessages(lapply(c(pkg), library, character.only = TRUE)))

# Data import

In [32]:
#traindir <- here('code', 'dev', 'data')

traindir <- here('data', 'svm')

pos <- c('E105H_ATAC', 'E115H_ATAC', 'ENCSR451NAE', 'ENCSR552ABC' ,'ENCSR652CNN','ENCSR068YGC')
neg <- c('G4-LIF','ENCSR088UYE', 'ENCSR154BXN', 'ENCSR559FAJ', 'ENCSR302LIV', 'ENCSR551WBK')

## load pos & neg pk sets in as lists
pos <- lapply(pos, function(file){
    path <- file.path(traindir, 'bed', sprintf('%s.mm10.narrowPeak', file))
    rtracklayer::import(path,format='narrowPeak')
})

neg <- lapply(neg, function(file){
    path <- file.path(traindir, 'bed',sprintf('%s.mm10.narrowPeak', file))
    rtracklayer::import(path,format='narrowPeak')
})

## load shared invervals generated by bedtools

pos_inter <- import.bed(file.path(traindir,'bed','pos_set_multiinter.mm10.bed'))
#neg_inter <- import.bed(file.path(traindir,'neg_set_multiinter.mm10.bed'))

## Filter peak sets (pos)
- filter peak sets (for positive set)
    - first filter out unique peaks from each sample
    - for multiple overlapping peaks, keeping highest 'score

In [ ]:
pos_unique <- do.call('c', lapply(pos, function(gr){
    suppressWarnings(gr[!overlapsAny(gr, pos_inter)])
}))

#length=27848

pos_pooled <- suppressWarnings(do.call('c', pos))
    pos_pooled <- pos_pooled[overlapsAny(pos_pooled, pos_inter)]
ol <- findOverlaps(pos_inter, pos_pooled)

#length(unique(queryHits(ol))) #242457

#tic()
pos_keep <- unique(do.call('c', bplapply(unique(queryHits(ol)), function(i){
    ol_idx <- subjectHits(ol)[which(queryHits(ol)==i)]
    if(length(ol_idx) > 1){
        keep <- ol_idx[which.max(mcols(pos_pooled[ol_idx])$signalValue)]
        return(keep)
    }
},  BPPARAM=MulticoreParam(workers = 50))))

#toc() 217.578s

pos_final <- c(pos_unique, pos_pooled[pos_keep]) 

#length(pos_final) #87239

- filtering for neg set not necessary ⬇️

In [ ]:
# neg_unique <- do.call('c', lapply(neg, function(gr){
#     suppressWarnings(gr[!overlapsAny(gr, neg_inter)])
# }))

# neg_pooled <- suppressWarnings(do.call('c', neg))
#     neg_pooled <- neg_pooled[overlapsAny(neg_pooled, neg_inter)]
# ol2 <- findOverlaps(neg_inter, neg_pooled)

# ## for multiple overlap, get peak with highest signal score
# #tic()
# neg_keep <- unique(do.call('c', bplapply(unique(queryHits(ol)), function(i){
#     ol_idx <- subjectHits(ol2)[which(queryHits(ol2)==i)]
#     if(length(ol_idx) > 1){
#         keep <- ol_idx[which.max(mcols(neg_pooled[ol_idx])$signalValue)]
#         return(keep)
#     }
# },  BPPARAM=MulticoreParam(workers = 50))))

# #toc() #202.9

# neg_final <- c(neg_unique, neg_pooled[neg_keep])

# #length(neg_final) #154187

# ## trim neg set to 500bp by summit
# start(neg_final) <- start(neg_final) + neg_final$peak - 250
# neg_final <- resize(neg_final, 500, 'start')

# ## neg set = 2kb away from any pos peak
# neg_final <- neg_final[!overlapsAny(neg_final, 
#                                     resize(pos_final, 2000, 'center'))]


# length(neg_final) #111852
# ## subsample to get same number of peak between 2 sets
# #set.seed <- 611
# #neg_final.ss <- sample(neg_final, length(pos_final))

- filter pos set to be outside 2kb of annotated TSS (from EPD3, mm10), i.e. prioritize enhancer-like signatures

In [ ]:
prom <- resize(import.bed(here('_data', 'references', 'mm10_epd3.bed')), width = 1000, fix = 'center')

## trim pos_final to 500bp around summit
start(pos_final) <- start(pos_final) + pos_final$peak - 250
pos_final <- resize(pos_final, 500, 'start')

pos_final <- pos_final[!overlapsAny(pos_final, prom, maxgap = 2000)]

#length(pos_final) #64136

In [14]:
export.bed(pos_final, 
          file.path(traindir,'bed', 'pos_set_filtered_500bp.mm10.bed'))

## Generate null seqs

- generate matching GC & repeats null seqs from the genome & get those overlapping with neg peaks as final negset
- adapt genNullseq function

In [17]:
## adapted from https://rdrr.io/cran/gkmSVM/src/R/genNullSeqs.R

genNullSeqs = function(
  inputBed, 
  xfold = 1, 
  repeat_match_tol = 0.02, 
  GC_match_tol = 0.02, 
  length_match_tol = 0.02, 
  batchsize = 5000, 
  nMaxTrials = 20, 
  genome = NULL   
){
  
  
  if (requireNamespace("GenomicRanges", quietly = TRUE)&
      requireNamespace("rtracklayer", quietly = TRUE)&
      requireNamespace("BiocGenerics", quietly = TRUE)&
      requireNamespace("Biostrings", quietly = TRUE)&
      requireNamespace("GenomeInfoDb", quietly = TRUE)&
      requireNamespace("IRanges", quietly = TRUE)&
      requireNamespace("S4Vectors", quietly = TRUE)){
    
    
    #inputBedFile = '~/Downloads/ctcfpos.bed'
    #xfold = 1; 
    # if(is.null(genome)){  
    #   if(toupper(genomeVersion)=='HG18'){
    #     if(requireNamespace("BSgenome.Hsapiens.UCSC.hg18.masked", quietly = TRUE)){
    #       genome <- BSgenome.Hsapiens.UCSC.hg18.masked::BSgenome.Hsapiens.UCSC.hg18.masked
    #     }
    #   } else{
    #     if(requireNamespace("BSgenome.Hsapiens.UCSC.hg19.masked", quietly = TRUE)){
    #       genome <- BSgenome.Hsapiens.UCSC.hg19.masked::BSgenome.Hsapiens.UCSC.hg19.masked
    #     }
    #   }
    # } else {
        #genome <- BSgenome.Sscrofa.UCSC.susScr3.masked::BSgenome.Sscrofa.UCSC.susScr3.masked
    # }
    
    seqnams= GenomeInfoDb::seqnames(genome)
    chrlens = GenomeInfoDb::seqlengths(genome)
    chrpos = cumsum(as.numeric(chrlens))
    pmax = max(chrpos)
    chrpos = c(chrpos,1E12)
    chrpos0 = c(0, chrpos)
    ichrA = as.character(names(chrlens)); 
    
    
    getichrpos = function(ipos){
      j = order(ipos); 
      ipos = sort(ipos); 
      ci = 1;
      res = rep(NA, length(ipos))
      for(i in 1:length(ipos))
      {
        while(ipos[i]>chrpos[ci]){
          ci = ci+1; 
        }
        res[j[i]] = ci
      }
      return(res); 
    }
    
    generateRandomGenSeqs = function(seqlens){
      rpos = sample(pmax, length(seqlens), replace = TRUE)
      ichr1 = getichrpos(rpos)
      ichr2 = getichrpos(rpos+seqlens)
      
      jj = which(ichr1!=ichr2)
      while(length(jj)>0){
        rpos[jj] = sample(pmax, length(jj), replace = TRUE)
        ichr1 = getichrpos(rpos)
        ichr2 = getichrpos(rpos+seqlens)
        jj = which(ichr1!=ichr2)
      }
      chr = ichrA[ichr1]
      start = rpos - chrpos0[ichr1];
      names <- chr; 
      ranges <- IRanges::IRanges(start=start, width=seqlens)
      strand <- BiocGenerics::strand(sample(c("+", "-"), length(names), replace=TRUE))
      gr <- GenomicRanges::GRanges(seqnames=names, ranges=ranges, strand=strand)
    }
    
    #inBed = rtracklayer::import.bed(inputBedFN)
     inBed = inputBed 
    
    #check the BED file:
    inbed = GenomicRanges::as.data.frame(inBed)
    jj = which(is.na(match(as.character(inbed$seqnames),as.character(seqnams))))
    if (length(jj)>0){
      cat( paste('ERROR: Chromosome name not recognized for',length(jj), 'sequences.\n'))
      cat( unique(as.character(inbed$seqnames[jj])))
      return(NULL)
    }
    jj = which(inbed$end>GenomeInfoDb::seqlengths(genome)[as.character(inbed$seqnames)])
    if (length(jj)>0){
      cat( 'ERROR: Region outside chromosome. (Check the genome version) \n')
      print( inbed[jj,])
      return(NULL)
    }
    
    gcContent <- function(seqs)
    {
      alf <- Biostrings::alphabetFrequency(seqs, as.prob=TRUE)
      gc = rowSums(alf[,c("G", "C"), drop=FALSE])
    }
    # 
    # gc1=desGC[unmatched]
    # gc2=rndGC
    # len1=desLens[unmatched]
    # len2=width(rndBed)
    # rpt1=desRpt[unmatched]
    # rpt2=rndRpt
    # gc_th = GC_match_tol,
    # len_th = repeat_match_tol,
    # rpt_th = length_match_tol)
    
    matchSeqs = function(gc1, gc2, len1, len2, rpt1, rpt2,  gc_th=0.02, len_th=0.02, rpt_th=0.02){
      ##
      #  gc1 = rndGC; 
      #  gc2 = inGC; 
      #  len1 = seqlens; 
      #  len2 = seqlens;
      #  rpt1 = inRpt; 
      #  rpt2 = rndRpt; 
      #  gc1=desGC[unmatched];gc2= rndGC;len1= desLens[unmatched];len2= BiocGenerics::width(rndBed);rpt1= desRpt[unmatched];rpt2= rndRpt;
                      
      #  gc_th=0.02
      #  len_th=0.02
      
      
      # gc1, len1 are the desired 
      # gc2 and len2 are to be matched 
      len_th = len_th * len1; 
      
      i1 = order(gc1)
      i2 = order(gc2)
      
      gc1 = gc1[i1]
      gc2 = gc2[i2]
      len1 = len1[i1]
      len2 = len2[i2]
      rpt1 = rpt1[i1]
      rpt2 = rpt2[i2]
      
      gc2 = c(gc2, 1E10)
      
      len_th = len_th[i1]
      
      m2 = 1; 
      N = length(i1); 
      N2 = length(i2);
      mtc1 = rep(NA, N)
      mtc2 = rep(0, length(i2))
      for(i in 1:N){
        #if(i%%1000==0){cat(i,' ')}
        gc1i = gc1[i]; 
        len1i = len1[i]
        rpt1i = rpt1[i]
        len_thi = len_th[i]
        
        while(gc1i - gc2[m2]>gc_th) {
          m2 = m2+1; 
        }
        if(m2<=N2){
          m2b=m2;
          while(gc2[m2b]-gc1i<=gc_th){
            if ((mtc2[m2b]==0)&(abs(len1i-len2[m2b])<=len_thi)&(abs(rpt1i-rpt2[m2b])<=rpt_th)){
              mtc2[m2b]=i; 
              mtc1[i]=m2b;
              if(m2b==m2){m2 = m2+1;}
              break; 
            }
            m2b =m2b+1; 
          }
        }else{break;}
      }
      
      mtc1 = i2[mtc1]
      res = rep(NA, N)
      res[i1] = mtc1; 
      return(res)
    }
    
    #bed = rndBed
    repeatRat = function(bed){
      chrs = unique(GenomeInfoDb::seqnames(bed))
      rpts = rep(0, length(bed))
      
      for(ichr in as.character(chrs)){
        seq = genome[[ichr]]
        rpt = Biostrings::masks(seq)[['TRF']]
        
        jj = which(as.character(GenomeInfoDb::seqnames(bed))==ichr)
#       jj = which((GenomeInfoDb::seqnames(bed))==ichr)
        
        if (length(jj)>0){
          
          jbed = bed[jj]
          jrpts = rep(0, length(jj))
          
          olaps <- IRanges::findOverlaps(rpt, jbed@ranges)
          #isect <- pintersect(rpt[queryHits(olaps)], jbed@ranges[subjectHits(olaps)])
          
          qdf= GenomicRanges::as.data.frame(rpt)[S4Vectors::queryHits(olaps),]
          isect <- IRanges::pintersect(IRanges::IRanges(start=qdf$start,end=qdf$end), jbed@ranges[S4Vectors::subjectHits(olaps)])
          
          jres = S4Vectors::subjectHits(olaps)
          olap_width=BiocGenerics::width(isect)
          
          ## this could be done faster if duplicated(jres) is empty 
          for(i in 1:length(jres)){
            jrpts[jres[i]]=jrpts[jres[i]]+olap_width[i]
          } 
          rpts[jj] = jrpts;
        }
      }
      
      rpts = rpts/BiocGenerics::width(bed)
    }
    
    #check the BED file:
    inbed = GenomicRanges::as.data.frame(inBed)
    
    cat(' importing sequences from', GenomeInfoDb::bsgenomeName(genome),'\n')
    #extract sequences
    inSeqs = Biostrings::getSeq(genome, inBed)
    seqlens = inbed$width
    inGC = gcContent(inSeqs)
    cat(' calculating repeat distributions\n')
    inRpt = repeatRat(inBed)
    
    nout = round(nrow(inbed)*xfold)
    #outbed=as.data.frame(matrix(ncol=ncol(inbed), nrow=nout))
    outbed=matrix(ncol=ncol(inbed), nrow=nout)
    outSeq = rep(inSeqs, length=nout); 
    
    colnames(outbed)=colnames(inbed)
    
    unmatched = 1:length(outSeq)
    
    desGC = rep(inGC, length=nout); #desired output GC
    desRpt = rep(inRpt, length=nout); #desired output repeat
    desLens = rep(seqlens, length=nout); #desired output lengths 
    
    for(iter in 1:nMaxTrials){
      if(length(unmatched)>0){
        cat(' Trial',iter,'out of',nMaxTrials,'\n')
        rndBed = generateRandomGenSeqs(rep(desLens[unmatched],length.out=batchsize))
        rndbed= GenomicRanges::as.data.frame(rndBed)
        cat(' importing sequences\n')
        rndSeqs = Biostrings::getSeq(genome, rndBed)
        rndGC = gcContent(rndSeqs)
        cat(' calculating repeat distributions\n')
        rndRpt = repeatRat(rndBed)
        cat(' matching sequences\n')
        
        mtc = matchSeqs(desGC[unmatched], rndGC, desLens[unmatched], BiocGenerics::width(rndBed), desRpt[unmatched], rndRpt,
                        gc_th = GC_match_tol,
                        len_th = length_match_tol,
                        rpt_th = repeat_match_tol)
        jj = which(!is.na(mtc))
        if(length(jj)>0){
          #outbed[unmatched[jj],]=rndbed[mtc[jj],];
          outbed[unmatched[jj],1:5]=as.matrix(rndbed[mtc[jj],]);
          outSeq[unmatched[jj],]=rndSeqs[mtc[jj],];
          unmatched = unmatched[-jj]
        }
        cat(nrow(outbed) - length(unmatched),'sequences found so far, ',length(unmatched), ' remaining.\n')
      }
    }  
    
    if(length(unmatched)>0){
      outbed = outbed[-unmatched,]
      outSeq = outSeq[-unmatched,]
    }

    outbed = gsub(' ','', outbed)
    #utils::write.table(as.matrix(outbed[,1:3]),quote = FALSE, sep='\t',row.names = FALSE, col.names = FALSE , file = outputBedFN)  
    # if(requireNamespace("seqinr", quietly = TRUE)){
    #   outseqnams = paste(outbed[,1], outbed[,2], outbed[,3], 'neg', 1:nrow(outbed), sep='_')
    #   seqinr::write.fasta(sequences = sapply(as.character(outSeq), strsplit,''), names =outseqnams,file.out =   outputNegFastaFN); 
    #   inseqnams = paste(as.character(inbed[,1]), inbed[,2], inbed[,3], 'pos', 1:nrow(inbed), sep='_')
    #   seqinr::write.fasta(sequences = sapply(as.character(inSeqs), strsplit,''), names =inseqnams, file.out =   outputPosFastaFN); 
    # }
    return(outbed)
    
      }else{
        cat('\n "GenomicRanges" and "rtracklayer" packages are needed. \n')
      }
}

In [ ]:
library(BSgenome.Mmusculus.UCSC.mm10.TRF.masked)

In [24]:
#tic()
matchingNull_bed <- bplapply(split(pos_final, seqnames(pos_final)), function(chrom){
    genNullSeqs(inputBed = chrom, 
                genome = BSgenome.Mmusculus.UCSC.mm10.TRF.masked::BSgenome.Mmusculus.UCSC.mm10.TRF.masked, 
                xfold = 10)
}, BPPARAM=MulticoreParam(workers = 43))
#toc() #564.433 sec elapsed



matchingNull_bed <- data.frame(do.call('rbind', unname(matchingNull_bed)))

#nrow(matchingNull_bed ) #481328

- convert to GRanges

In [30]:
matchingNull_gr <- GRanges(seqnames = matchingNull_bed$seqnames,
                           ranges = IRanges(start = as.numeric(matchingNull_bed$start), width = 500), 
                           strand = rep('*', nrow(matchingNull_bed)), 
                           name = paste0('RND_', seq_along(matchingNull_bed$name)))

In [31]:
head(matchingNull_gr)

GRanges object with 6 ranges and 1 metadata column:
      seqnames              ranges strand |        name
         <Rle>           <IRanges>  <Rle> | <character>
  [1]    chr10   93821860-93822359      * |       RND_1
  [2]     chr2 138144687-138145186      * |       RND_2
  [3]    chr14   32990543-32991042      * |       RND_3
  [4]    chr15   75736095-75736594      * |       RND_4
  [5]    chr17   42677890-42678389      * |       RND_5
  [6]     chr4   22504669-22505168      * |       RND_6
  -------
  seqinfo: 236 sequences from an unspecified genome; no seqlengths

- After filtering out null-seq within 2kb of pos peaks, options are:
    - (1) get those overlapping with neg ATAC peaks from limb, brain, G4
    - (2) get those overlapping with ENCODE cCREs
    - (3) because ENCODE cCRES did not include embryonic data, combine (1) & (2) !![THIS]

In [56]:
ccres <- import.bed(here('_data', 'references', 'ENCODE_cCRE.mm10.bed'))
#length(ccres) #368121
neg_ccre <- c(neg, ccres)

In [ ]:
null_final <- suppressWarnings(matchingNull_gr[!overlapsAny(matchingNull_gr, pos_final, maxgap = 2000)])
#length(null_final) #403041
null_final <- suppressWarnings(null_final[overlapsAny(null_final, neg_ccre, minoverlap = 0.1*500)])
#length(null_final) #67218

-----

In [63]:
export.bed(pos_final, 
          file.path(traindir,'bed', 'pos_set_filtered_500bp.mm10.bed'))

export.bed(null_final, 
          file.path(traindir,'bed','neg_set_GC-rpt-matched_500bp.mm10.bed'))

In [41]:
set.seed <- 666
test.neg <- sample(neg_pooled, 15)

- instead of chrom/seqlengths as max length, take width of each pk as max

In [ ]:
library(BSgenome.Mmusculus.UCSC.mm10)

mm10 <- BSgenome.Mmusculus.UCSC.mm10

In [53]:
getichrpos = function(ipos){
  j = order(ipos); 
  ipos = sort(ipos); 
  ci = 1;
  res = rep(NA, length(ipos))
  for(i in 1:length(ipos))
  {
    while(ipos[i]>chrpos[ci]){
      ci = ci+1; 
    }
    res[j[i]] = ci
  }
  return(res); 
}

In [54]:
rpos <- sample(pmax, 20)

In [56]:
getichrpos(rpos)

[1] 16  5 12  5 14 14  7 13  8 13 20  5  7 18  8  3 15  7  6 17

In [52]:
sample(pmax, 5)

[1] 1110742384  257091909  714336825 1408754075 2650870912

In [46]:
cumsum(width(test.neg[1]))

[1] 971